In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,Flatten,MaxPool2D,Reshape,InputLayer
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [4]:
IMAGE_SIZE = [224,224]
path = 'Digital Naturalist Dataset'

In [5]:
train_data_gen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             validation_split = 0.30)
test_data_gen = ImageDataGenerator(rescale = 1./255,validation_split = 0.30)

In [6]:
training_set = train_data_gen.flow_from_directory(path,
                                                 target_size=(224,224),
                                                 batch_size=100,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 color_mode='rgb',
                                                 subset = 'training')

testing_set = test_data_gen.flow_from_directory(path,
                                                 target_size=(224,224),
                                                 batch_size=100,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 color_mode='rgb',
                                                 subset = 'validation')

Found 17270 images belonging to 142 classes.
Found 7382 images belonging to 142 classes.


In [7]:
training_set.class_indices

{'Apple Leaf': 0,
 'Arrowhead Leaf': 1,
 'Bamboo Leaf': 2,
 'Banana Leaf': 3,
 'Bluebell': 4,
 'Buttercup': 5,
 "Colts'foot": 6,
 'Corals': 7,
 'Corpse Flower': 8,
 'Cowslip': 9,
 'Crabs': 10,
 'Crocus': 11,
 'Currant Leaf': 12,
 'Curry Leaf': 13,
 'Daffodil': 14,
 'Dolphin': 15,
 'Duckpotato Leaf': 16,
 'Eel': 17,
 'Fig Leaf': 18,
 'Fritillary': 19,
 'Gooseberry Leaf': 20,
 'Great Indian Bustard Bird': 21,
 'Guava Leaf': 22,
 'Iris': 23,
 'Irrelevant': 24,
 'Jelly Fish': 25,
 'Lady Slipper Orchid Flower': 26,
 'Lemon Leaf': 27,
 'Lily Valley': 28,
 'Lobster': 29,
 'Mango Leaf': 30,
 'Mulberry Leaf': 31,
 'Nudibranchs': 32,
 'Octopus': 33,
 'Pangolin Mammal': 34,
 'Pansy': 35,
 'Papaya Leaf': 36,
 'Pine Leaf': 37,
 'Puffers': 38,
 'Raddish Leaf': 39,
 'Raspberry Leaf': 40,
 'Rose Leaf': 41,
 'Sea Rays': 42,
 'Sea Urchins': 43,
 'Seahorse': 44,
 'Seal': 45,
 'Senenca White Deer Mammal': 46,
 'Sharks': 47,
 'Snowdrop': 48,
 'Spoon Billed Sandpiper Bird': 49,
 'Squid': 50,
 'Starfish': 51

In [8]:
vgg19 = VGG19(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 19s 0us/step


In [9]:
for layer in vgg19.layers:
    layer.trainable = False

In [10]:
from glob import glob
folders = glob('Digital Naturalist Dataset/*')

In [11]:
len(folders)

142

In [12]:
x = Flatten()(vgg19.output)
prediction = Dense(len(folders),activation='softmax')(x)
model = Model(inputs=vgg19.input, outputs=prediction)

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
early_stop = EarlyStopping(monitor='val_accuracy', 
                           patience=5,verbose=1,mode='auto')

lr = ReduceLROnPlateau(monitor='val_accuracy', 
                       factor=0.2,patience=5, 
                       min_lr=0.00001)

callback = [early_stop,lr]

In [ ]:
result = model.fit_generator(training_set, validation_data=testing_set, epochs=50,steps_per_epoch=len(training_set),
  validation_steps=len(testing_set),callbacks=callback)

In [ ]:
#plot the loss
plt.plot(result.history['loss'], label='train loss')
plt.plot(result.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# plot the accuracy
plt.plot(result.history['accuracy'], label='train acc')
plt.plot(result.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
model.save('dnai.h5')